<a href="https://colab.research.google.com/github/kunwarsharik/SharikProjects/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os,json,nltk,gensim,numpy as np,pickle
from gensim import corpora, models, similarities

In [3]:
!wget https://raw.githubusercontent.com/shreyans29/thesemicolon/master/conversation.json

--2020-01-30 17:58:34--  https://raw.githubusercontent.com/shreyans29/thesemicolon/master/conversation.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881 (3.8K) [text/plain]
Saving to: ‘conversation.json’

conversation.json   100%[===================>]   3.79K  --.-KB/s    in 0s      

2020-01-30 17:58:34 (104 MB/s) - ‘conversation.json’ saved [3881/3881]



In [4]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
!gunzip GoogleNews-vectors-negative300.bin.gz

--2020-01-30 17:58:39--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.15.30
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.15.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  79.9MB/s    in 22s     

2020-01-30 17:59:01 (71.3 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [5]:
model=gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
f=open('conversation.json')
data = json.load(f)
cor=data["conversations"]

In [7]:
nltk.download('punkt')
x,y=[],[]
for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j])
            y.append(cor[i][j+1])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
tx,ty=[],[]
for i in range(len(x)):
    tx.append(nltk.word_tokenize(x[i].lower()))
    ty.append(nltk.word_tokenize(y[i].lower()))

In [0]:
sentend=np.ones((300,),dtype=np.float32)

In [0]:
vx,vy=[],[]#vector for x
for sent in tx:
    vx.append([model[w] for w in sent if w in model.vocab])
for sent in ty:
    vy.append([model[w] for w in sent if w in model.vocab])

In [0]:
for ts in vx:
    ts[14:]=[]
    ts.append(sentend)
for ts in vx:
    if len(ts)<15:
        for i in range(15-len(ts)):
            ts.append(sentend)

In [0]:
for ts in vy:
    ts[14:]=[]
    ts.append(sentend)
for ts in vy:
    if len(ts)<15:
        for i in range(15-len(ts)):
            ts.append(sentend)

In [0]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
from sklearn.model_selection import train_test_split as tt

In [0]:
vx=np.array(vx)
vy=np.array(vy)

In [15]:
vx.shape

(86, 15, 300)

In [0]:
x_train,x_test, y_train,y_test = tt(vx, vy, test_size=0.2, random_state=1)

In [16]:
x_train.shape

(68, 15, 300)

In [17]:
model2=Sequential()
model2.add(LSTM(output_dim=300,input_shape=(15,300),return_sequences=True, init='glorot_normal', inner_init='glorot_normal', activation='sigmoid'))
model2.add(LSTM(output_dim=300, input_shape=(15,300),init='glorot_normal', inner_init='glorot_normal',activation='sigmoid',return_sequences=True))
model2.add(LSTM(output_dim=300, input_shape=(15,300), init='glorot_normal', inner_init='glorot_normal',activation='sigmoid',return_sequences=True))
model2.add(LSTM(output_dim=300,input_shape=(15,300),init='glorot_normal', inner_init='glorot_normal', activation='sigmoid',return_sequences=True))
model2.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])
model2.fit(x_train, y_train, nb_epoch=200,validation_data=(x_test, y_test))
predictions=model2.predict(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), activation="sigmoid", return_sequences=True, units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), activation="sigmoid", return_sequences=True, units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  after removing the cwd from sys.path.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), activation="sigmoid", return_sequences=True, units=300, kernel_initializer="glorot_normal", recurrent_initializer="glorot_normal")`
  """


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys





Train on 68 samples, validate on 18 samples
Epoch 1/200





68/68 [==============================] - 12s 174ms/step - loss: -0.6461 - acc: 0.0088 - val_loss: -0.6194 - val_acc: 0.0296
Epoch 2/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6547 - acc: 0.0118 - val_loss: -0.6235 - val_acc: 0.0185
Epoch 3/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6585 - acc: 0.0059 - val_loss: -0.6266 - val_acc: 0.0185
Epoch 4/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6625 - acc: 0.0059 - val_loss: -0.6295 - val_acc: 0.0148
Epoch 5/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6656 - acc: 0.0078 - val_loss: -0.6329 - val_acc: 0.0074
Epoch 6/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6686 - acc: 0.0029 - val_loss: -0.6361 - val_acc: 0.0111
Epoch 7/200
68/68 [==============================] - 0s 5ms/step - loss: -0.6713 - acc: 0.0039 - val_loss: -0.6392 - val_acc: 0.0111
Epoch 8/200
68

In [41]:
[model.most_similar([predictions[10][i]])[0] for i in range(15)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('supose', 0.503160297870636),
 ('is', 0.5825797319412231),
 ('do', 0.531836211681366),
 ('Circumstances_dictate', 0.4936249554157257),
 ("l'_Affaire", 0.257721483707428),
 ("l'_Affaire", 0.241354301571846),
 ("l'_Affaire", 0.23994538187980652),
 ("l'_Affaire", 0.23784194886684418),
 ("l'_Affaire", 0.2372058629989624),
 ("l'_Affaire", 0.2370615154504776),
 ("l'_Affaire", 0.2369934618473053),
 ("l'_Affaire", 0.23697099089622498),
 ("l'_Affaire", 0.23696526885032654),
 ("l'_Affaire", 0.23700056970119476),
 ("l'_Affaire", 0.23700818419456482)]

In [42]:
predictions.shape

(18, 15, 300)

In [46]:
[model.most_similar([y_test[11][i]])[0] for i in range(15)]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('beautiful', 1.0),
 ('is', 1.0000001192092896),
 ('better', 1.0),
 ('than', 1.0),
 ('ugly', 1.0),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936),
 ("l'_Affaire", 0.24067771434783936)]

In [0]:
#model2.save('./drive/My Drive/LSTM chatbot.h5')
#os.path.exists

In [50]:
for _ in range(3):
    mssg=input("Enter the message:")
    x=mssg#x_test[0]
    sentend=np.ones((300,),dtype=np.float32)
    sent=nltk.word_tokenize(x.lower())
    sentvec = [model[w] for w in sent if w in model.vocab]

    sentvec[14:]=[]
    sentvec.append(sentend)
    if len(sentvec)<15:
        for i in range(15-len(sentvec)):
            sentvec.append(sentend) 

    sentvec=np.array([sentvec])
    predictions = model2.predict(sentvec)
    print([model.most_similar([predictions[0][i]])[0][0] for i in range(15)])

Enter the message:hello i am boy


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['supose', 'is', "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire"]
Enter the message:how are you?
['supose', 'is', 'do', 'do', 'RUSH_Okay', "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire"]
Enter the message:thank you
['supose', 'is', "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire", "l'_Affaire"]


In [37]:
xp=x_test[1].reshape(1,15,300)
yp=y_test[1].reshape(1,15,300)
pred=model2.predict(xp)
print("prediction ",model.most_similar(pred[0]))
print("actual ",model.most_similar(yp[0]))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


prediction  [('wish_washy', 0.2750976085662842), ("l'_Affaire", 0.2700481414794922), ('reallllly', 0.261417031288147), ('relevations', 0.25948765873908997), ('aide_Melanie_Fonder', 0.24947303533554077), ('utterly_repulsive', 0.24779541790485382), ('constitutional_amendments', 0.24728330969810486), ('Australian_sheepskin_boots', 0.24708551168441772), ('Cindy_Crawford', 0.24577397108078003), ('opera_Elaine_Padmore', 0.24496468901634216)]
actual  [("l'_Affaire", 0.2522413730621338), ('relevations', 0.23910337686538696), ('Ovidiu_Rom', 0.23834006488323212), ('Cindy_Crawford', 0.2333430051803589), ('Australian_sheepskin_boots', 0.23041939735412598), ('opera_Elaine_Padmore', 0.22765450179576874), ('constitutional_amendments', 0.22741520404815674), ('AP_HOCKEY_NEWS', 0.22420454025268555), ('lawsuit_Patricia_Conradt', 0.22418904304504395), ('aide_Melanie_Fonder', 0.22354507446289062)]
